<a href="https://colab.research.google.com/github/juanfranbrv/curso-langchain/blob/main/Prompts%20e%20Ingenier%C3%ADa%20de%20Prompts%20en%20LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prompts e Ingeniería de Prompts dentro de LangChain  
---
Este cuaderno se enfoca en los **Prompts** y en la **Ingeniería de Prompts** dentro de LangChain. Abordaremos los fundamentos de los prompts, por qué son tan importantes y cómo LangChain nos ayuda a crearlos y reutilizarlos de manera efectiva con distintas clases de Templates: `PromptTemplate`, `FewShotPromptTemplate` y `ChatPromptTemplate`.

## 1. Introducción a los Prompts

En el contexto de los grandes modelos de lenguaje (Large Language Models - LLMs), el prompt es el mensaje, instrucción o pregunta que proporcionamos para guiar la respuesta del modelo, por ejemplo:

-  “Explícame en tres frases qué es el *Machine Learning*”.

Cuando diseñamos prompts, buscamos que el modelo entienda el contexto y la tarea específica que queremos resolver.

Un buen prompt permite orientar al LLM para que genere una respuesta más precisa y alineada con la tarea.  

Mediante técnicas como *few-shot prompting*, *Chain-of-Thought (CoT) Prompting* y otras, podemos moldear la personalidad y estilo de la respuesta.  

Los prompts son fundamentales en el manejo de modelos de lenguaje ya que guían al modelo para generar respuestas coherentes y útiles. Un prompt bien diseñado puede marcar la diferencia entre una salida precisa y relevante, y una respuesta confusa o fuera de contexto.  

LangChain ofrece herramientas específicas para gestionar y optimizar prompts de manera eficiente. Estas herramientas permiten crear plantillas de prompts reutilizables, adaptarlas dinámicamente según el contexto y personalizarlas para diferentes aplicaciones.


## 2. Configuración del entorno del cuaderno

Instalamos los paquetes necesarios para LangChain, incluyendo soporte para OpenAI y Groq, y recupera las claves API correspondientes desde los secretos de Colab para fines de autenticación.
(Este codigo se explico con detalle en el cuaderno anterior)



In [ ]:
%%capture --no-stderr
%pip install langchain -qU

%pip install langchain-openai -qU
%pip install langchain-groq -qU
# %pip install langchain-google-genai -qU
# %pip install langchain-huggingface -qU

from google.colab import userdata

OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')
GROQ_API_KEY=userdata.get('GROQ_API_KEY')
# GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
# HUGGINGFACEHUB_API_TOKEN=userdata.get('HUGGINGFACEHUB_API_TOKEN')


## 3. PromptTemplate
---
Los prompt templates en Langchain permiten crear prompts dinámicos y flexibles mediante la incorporación de variables y opciones de formato, lo que le permite personalizar los prompts en función de los datos de entrada o tareas específicas.

- Ayudan en la construccion de prompts complejos.
- Son dinamicos: tienen *placeholders* para variables {variable}  
- Pueden reutilizarse  

Imagina que quieres reutilizar la misma estructura de prompt para múltiples entradas; en lugar de escribir el prompt completo cada vez, puedes definir una “plantilla” con huecos que se llenan de forma programática.

Ejemplos:

- "Explicame {tema} con nivel de dificultad {nivel}"

- "Traduce {frase} del {idioma_entrada} a {idioma_salida}"

Prompts como los anteriores son genericos y reutilizables, pues se usan com plantillas para la construcción de prompts concretos.

### Sintaxis básica

`PromptTemplate` es la clase base que nos permite crear plantillas de prompts con variables dinámicas.



```
prompt_template = PromptTemplate(
            input_variables=["variable1", "variable2"],  # Lista de variables dinámicas
            template="Este es un ejemplo donde {variable1} y {variable2} son variables dinámicas."
)
```



*Observación: De momento no vamos a consultara ningún LLM. Sólo estamos construyendo un prompt que luego podriamos pasar a un modelo.*

1. Definimos una cadena de texto (**`plantilla_texto`**) con dos variables dinámicas: `{tema}` y `{pregunta}`.

2. Usamos el contructor de clase `PromptTemplate()` para crear un objeto PromptTemplate indicando cuáles son esas variables (input_variables) y cuál es la plantilla (template).

3. Luego usamos el metodo `.format()` de la clase, para rellenar los huecos del template.

In [ ]:
from langchain import PromptTemplate

plantilla_texto = """
Eres un experto en {tema}.
Por favor, responde la siguiente pregunta con detalle:
Pregunta: {pregunta}
"""

prompt_template = PromptTemplate(
    input_variables=["tema", "pregunta"],
    template=plantilla_texto
)

prompt=prompt_template.format(tema="programacion", pregunta="Que es python?")

prompt

La clase `PromptTemplate` tiene un metodo que permite instanciar objetos sin necesidad de usar el contructor de clase lo que permite que el codigo sea mas compacto y legible. Este metodo es `.from_template()`


```
prompt_template = "Hola, {nombre}. Hoy es {dia}. ¿Cómo estás?"
prompt = PromptTemplate.from_template(prompt_template)
```



`PromptTemplate.from_template()` es un _método de clase_ que permite crear una instancia de `PromptTemplate` de manera directa a partir de una cadena de texto que contiene variables de relleno (por ejemplo, `{tema}` y `{pregunta}`). A diferencia de usar el constructor estándar (donde se debe especificar `input_variables` y la plantilla por separado), esta función:

1. **Extrae automáticamente** los nombres de las variables desde las llaves (p. ej., `{tema}`, `{pregunta}`).
2. **Crea y retorna** un objeto `PromptTemplate` listo para usar, lo que hace el código más compacto y legible.   

Este es el metodo preferido para crear PromptTemplates

In [ ]:
from langchain_core.prompts import PromptTemplate

#Instanciar usando .from_template es la forma preferida
prompt_template = PromptTemplate.from_template("""
Eres un experto en {tema}.
Por favor, responde la siguiente pregunta con detalle:
Pregunta: {pregunta}
""")
prompt = prompt_template.format(tema="programación", pregunta="Que es python?")

prompt

Vamos ahora a crear un prompt template y enviarselo a un LLM


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq

llm1 = ChatOpenAI(model="gpt-4o-mini",api_key=OPENAI_API_KEY)
llm2 = ChatGroq(model="llama-3.3-70b-versatile", api_key=GROQ_API_KEY)

plantilla_texto = """
Eres un experto en traducción del {idioma_origen} al inglés.
Por favor, traduce el siguiente texto:
---
Texto: {texto_a_traducir}
---
Mantén la intención, el estilo y el tono original.
Proporciona solo el texto traducido, sin comentarios adicionales
"""

# Creamos el PromptTemplate usando las variables definidas
prompt_template = PromptTemplate(
    input_variables=["idioma_origen", "texto_a_traducir"],
    template=plantilla_texto
)

# Ejemplo de uso
prompt = prompt_template.format(
    idioma_origen="español",
    texto_a_traducir="¡Hola! ¿Cómo te va?"
)


#Invocamos los modelos pasandoles el prompt
respuesta1 = llm1.invoke(prompt)
respuesta2 = llm2.invoke(prompt)


# Extraemos de la respuesta el nombre del modelo y el resultado
print(respuesta1.response_metadata["model_name"]) # imprimimos el nombre del modelo
print(respuesta1.content)
print("---")
print(respuesta2.response_metadata["model_name"]) # imprimimos el nombre del modelo
print(respuesta2.content)


### Reto 1: Crea tu propio PromptTemplate

**Objetivo**  

Diseñar un `PromptTemplate` que reciba variables para traducir un texto de un idioma a otro, especificando además el estilo de traducción. Luego, pasar este prompt a un LLM para obtener el resultado y verificar su eficacia.

**Instrucciones**

1. **Crea un PromptTemplate** con **tres** variables:
    
    - `{idioma_origen}`: El idioma original en el que está el texto.
    - `{idioma_destino}`: El idioma al que se va a traducir.
    - `{texto_original}`: El texto que se quiere traducir.
2. **Incluye** en tu plantilla alguna instrucción sobre el **estilo** o **tono** de la traducción. Por ejemplo, podrías pedir que sea “formal” o “informal”.
    
3. **Crea** el prompt final usando `.format(...)` y pásaselo a un LLM de tu elección. Obtén la respuesta y **muéstrala** en pantalla.
    
4. (Opcional) **Experimenta** con distintos valores de _temperature_ o _top\_p_ (dependiendo del LLM que uses) para comparar la **consistencia** de la traducción.

In [ ]:
# Escribe aquí tu solución para el reto 1

### Solución Reto 1

In [ ]:
# Realmente no es necesario importar de nuevo las librerías.
# Solo lo hacemos con fines ilustrativos, para que el ejemplo sea autosuficiente.

from langchain import PromptTemplate
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama-3.3-70b-versatile", api_key=GROQ_API_KEY)

# 1. Define la plantilla con las variables {idioma_origen}, {idioma_destino}, y {texto_original}
mi_plantilla = """
Eres un traductor experto. Traducirás cuidadosamente del {idioma_origen} al {idioma_destino}.
Por favor, mantén un tono formal en la traducción.
---
Texto original: {texto_original}
---
"""

# 2. Crea el PromptTemplate
prompt_template = PromptTemplate(
    input_variables=["idioma_origen", "idioma_destino", "texto_original"],
    template=mi_plantilla
)

# 3. Usa .format para rellenar los huecos
prompt = prompt_template.format(
    idioma_origen="español",
    idioma_destino="francés",
    texto_original="¡Hola! ¿Cómo estás hoy?"
)

# 4. Invoca tu LLM y muestra el resultado
# (Ejemplo con un LLM hipotético `my_llm`)
response = llm.invoke(prompt, temperature=0.5)

print("Prompt final:\n", prompt)
print("\nTraducción:\n", response.content)


## 4. FewShotPromptTemplate: proporcionando ejemplos de referencia

**Few-shot prompting** es una técnica donde proporcionamos ejemplos dentro del prompt para que el modelo entienda mejor el contexto y la forma de la respuesta. Con `FewShotPromptTemplate` podemos manejar varios ejemplos y luego añadir la pregunta final.

### Sintaxis



```
FewShotPromptTemplate(
    examples=ejemplos,               # Lista de ejemplos
    example_prompt=example_prompt,   # PromptTemplate para cada ejemplo
    prefix=prefix_text,              # Texto que aparecerá antes de los ejemplos
    suffix=suffix_text,              # Texto que aparecerá después de los ejemplos, habitualmente la pregunta
    input_variables=["nuevo_input"], # Variables adicionales que usarás
    example_separator="\n---\n"      # Separador entre ejemplos
)
```



### ¿Cómo funciona FewShotPromptTemplate?

**¿Cómo funciona `FewShotPromptTemplate`?**

1. **`examples`:** Una lista de diccionarios. Cada diccionario representa un ejemplo y contiene pares clave-valor con las entradas y salidas deseadas para una tarea específica.
    
2. **`example_prompt`:** Una instancia de `PromptTemplate` que define cómo se formatea cada ejemplo individualmente. Define las variables de entrada que se utilizarán para cada ejemplo.
    
3. **`prefix`:** Una cadena que va al principio del prompt completo. Suele ser una instrucción general o una descripción de la tarea.
    
4. **`suffix`:** Una cadena que va al final del prompt completo. Normalmente aquí es donde se introduce la pregunta al modelo, que debria ser como los ejemplos, pero sin el resultado
    
5. **`input_variables`:** Una lista de nombres de variables que se utilizarán en el `suffix`.
    
6. **`example_separator`:** Una cadena que separa cada ejemplo en el prompt. Por defecto es `\n\n`.
    
    

**En resumen, `FewShotPromptTemplate` toma los ejemplos, los formatea usando `example_prompt`, los une con `example_separator`, y los coloca entre `prefix` y `suffix` para crear el prompt final. Este prompt contiene el contexto necesario (los ejemplos) para que el modelo de lenguaje pueda inferir la tarea y aplicarla a la nueva entrada proporcionada en el `suffix`.**

Hay que usar la sintaxis con el constructor de clase. No hay disponible ningun metodo que simplifique la sintaxis.

In [ ]:
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate

# Lista de ejemplo, donde cada ejemplo es un diccionario
ejemplos = [
    {"input": "Hello", "output": "Bonjour"},
    {"input": "Goodbye", "output": "Au revoir"},
    {"input": "Thank you", "output": "Merci"},
]

#Instancia de PromptTemplate para formatear los ejemplos
traduccion_prompt_template = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

# Creamos el prompt pasando:
# la lista de ejemplos
# el formateador de los ejemplos
# el prefix, que suele ser la descripcion general de la tarea
# el suffix, que donde introduciomes la pregunta con la variables
# La lista de variables


prompt = FewShotPromptTemplate(
    examples=ejemplos,
    example_prompt=traduccion_prompt_template,
    prefix="Translate the following English words to French:",
    suffix="Input: {english_word}\nOutput:",
    input_variables=["english_word"],
)

print(prompt.format(english_word="Car"))

Vamos a usarlo para realizar una llamada a los modelos que estamos usando en este cuaderno

In [ ]:
from langchain import FewShotPromptTemplate, PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq

# Creamos las instancias de cada modelo.
llm1 = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY)
llm2 = ChatGroq(model="llama-3.3-70b-versatile", api_key=GROQ_API_KEY)

# 1) Definimos algunos ejemplos de traducción: (input → output)
ejemplos = [
    {"input": "Buenos días", "output": "Good morning"},
    {"input": "¿Dónde está el baño?", "output": "Where is the bathroom?"}
]

# 2) Creamos un PromptTemplate para cada ejemplo: cómo se mostrará "input" y "output" en el prompt
traduccion_prompt_template = PromptTemplate(
    input_variables=["input", "output"],
    template="""
Ejemplo de Entrada: {input}
Ejemplo de Salida: {output}
"""
)

# 3) Definimos un prefijo y un sufijo para contextualizar la tarea antes y después de los ejemplos
prefix_text = """Eres un traductor experto del español al inglés.
Estos son algunos ejemplos de cómo traducir oraciones al inglés:"""

suffix_text = """
Ahora traduce la siguiente frase:
Entrada: {nueva_frase}
Salida:
"""

# 4) Creamos el FewShotPromptTemplate
few_shot_prompt = FewShotPromptTemplate(
    examples=ejemplos,                  # Nuestros ejemplos
    example_prompt=traduccion_prompt_template,      # Formato de cada ejemplo
    prefix=prefix_text,                 # Texto antes de los ejemplos
    suffix=suffix_text,                 # Texto después de los ejemplos
    input_variables=["nueva_frase"],    # Variable requerida al formatear
    example_separator="\n---\n"         # Separador entre ejemplos
)

# 5) Generamos el prompt final para la nueva frase
prompt = few_shot_prompt.format(nueva_frase="¡Hola! ¿Cómo te va?")

print("=== Prompt Generado ===")
print(prompt)

# 6) Pasamos el prompt al primer modelo (ChatOpenAI - gpt-4o-mini)
print("\n=== Respuesta con ChatOpenAI (gpt-4o-mini) ===")
response_llm1 = llm1.invoke(prompt)
print(response_llm1.content)

# 7) Pasamos el mismo prompt al segundo modelo (ChatGroq - llama-3.3-70b-versatile)
print("\n=== Respuesta con ChatGroq (llama-3.3-70b-versatile) ===")
response_llm2 = llm2.invoke(prompt)
print(response_llm2.content)


## Reto 2: Few-Shot Prompting para Traducción Creativa

El reto consiste en usar la clase FewShotPromptTemplate para traducir oraciones del español al inglés con un estilo específico, incorporando ejemplos previos que sirvan de guía para el LLM.  

### **Instrucciones**

1. **Crea una lista de ejemplos** de traducción que contengan:
    
    - La frase de entrada en español (clave: `"input"`).
    - Su traducción al inglés, **pero con cierto estilo o “sabor”** (clave: `"output"`).
        - Por ejemplo, podrías elegir que la traducción sea **muy “entusiasta”** o **tenga un tono humorístico**.
2. **Diseña un `PromptTemplate`** para mostrar cada ejemplo con un formato claro. Algo así como:
    
    python
    
    Copiar código
    
    `"Oración original: {input}\nTraducción estilizada: {output}\n"`
    
    (puedes ajustar el texto a tu preferencia).
    
3. **Define** el `prefix` y el `suffix` de tu `FewShotPromptTemplate`:
    
    - El `prefix` debe _explicar_ que el LLM es un traductor con cierto estilo (por ejemplo, “traductor humorístico” o “traductor formal”).
    - El `suffix` debe contener la **nueva frase** a traducir (`{frase_nueva}`) y especificar que se aplique el mismo estilo.
4. **Crea** la instancia de `FewShotPromptTemplate` con:
    
    - `examples`: tu lista de ejemplos.
    - `example_prompt`: tu plantilla de ejemplo.
    - `prefix`, `suffix`, `example_separator`, etc.
5. **Usa `.format()`** para generar el prompt final, pasando la nueva frase a traducir en la variable `{frase_nueva}`.
    
6. **Envía** ese prompt a un LLM (puede ser `ChatOpenAI`, `ChatGroq` o cualquier otro). Muestra la respuesta que obtienes.
    
7. (Opcional) **Experimenta** con distintos estilos (más formal, más bromista, etc.) para ver cómo cambian las traducciones.

8. (Opcional). Puedes incluir el estilo de traducción como una varibale dinámica del prompt ?

## Solución Reto 2

In [ ]:
from langchain import FewShotPromptTemplate, PromptTemplate

# 1. Ejemplos
ejemplos = [
    {
        "input": "Buenos días, ¿cómo amaneciste?",
        "output": "Good morning, how did you wake up, buddy? (super friendly tone)"
    },
    {
        "input": "¿Quieres salir a correr más tarde?",
        "output": "Do you want to go for a run later, pal? (energetic style)"
    }
]

# 2. Plantilla de ejemplo
ejemplo_plantilla = """
Oración original: {input}
Traducción con estilo: {output}
"""

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template=ejemplo_plantilla
)

# 3. Prefijo y Sufijo
prefix_text = """Eres un traductor que siempre usa un tono muy amigable y entusiasta.
Aquí tienes algunos ejemplos de cómo traduces del español al inglés:
"""
suffix_text = """
Ahora quiero que traduzcas la siguiente frase con el mismo tono amistoso:
Oración: {frase_nueva}
Traducción:
"""

# 4. FewShotPromptTemplate
few_shot_prompt = FewShotPromptTemplate(
    examples=ejemplos,
    example_prompt=example_prompt,
    prefix=prefix_text,
    suffix=suffix_text,
    input_variables=["frase_nueva"],
    example_separator="\n---\n"
)

# 5. Formateo del prompt
prompt_final = few_shot_prompt.format(frase_nueva="¡Hola! ¿Listo para la aventura de hoy?")

# 6. (Pseudo-código) Llamada al LLM
# response = llm.invoke(prompt_final)
# print(response.content)



## 5. ChatPromptTemplate: el modelo conversacional.
---

Los modelos de lenguaje más recientes (como `gpt-3.5-turbo` y `gpt-4`) trabajan muy bien en un _formato conversacional_. Esto significa que, en lugar de pasar un único “prompt de texto”, podemos estructurar el input como una serie de **mensajes** con distintos roles:

- **system** (mensaje de sistema)
- **user** (mensaje del usuario)
- **assistant** (mensaje del asistente)

`ChatPromptTemplate` facilita la construcción de estos mensajes, en lugar de tener que concatenarlos manualmente en un string.  

### **Componentes necesarios**

Para usar `ChatPromptTemplate`, necesitaremos:

1. **SystemMessagePromptTemplate**: Define las instrucciones de “sistema”, es decir, el contexto o la personalidad del asistente.
2. **HumanMessagePromptTemplate**: Representa el mensaje que proviene de un usuario.
3. (**Opcional**) **AIMessagePromptTemplate**: Sirve para simular respuestas del asistente (en caso de que queramos “inyectar” un mensaje del asistente en el contexto, aunque no siempre es necesario).

Finalmente, un **ChatPromptTemplate** agrupa todos estos “mensajes” para formar un “chat” que el modelo interpretará.

**Sintaxis**

Usando el contructor de clase:  
(aquí pasas la lista de mensajes directamente al parámetro messages del constructor.)
```
ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template("Mensaje del sistema"),
        HumanMessagePromptTemplate.from_template("Mensaje del humano"),
        AIMessagePromptTemplate.from_template("Mensaje de la IA"),
    ]
```
Usando el metodo `.from_messages`

```
ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("Mensaje del sistema"),
    HumanMessagePromptTemplate.from_template("Mensaje del humano"),
    AIMessagePromptTemplate.from_template("Mensaje de la IA"),  ])`
```

y tambien de esta forma mas legible, usando tuplas. Y nos evitamos crear instancias de SystemMessagePromptTemplate o HumanMessagePromptTemplate

```
chat_prompt_template = ChatPromptTemplate.from_messages([
    ("system", template_sistema),
    ("human", template_usuario),
])
```
Además, ten en cuenta que podemos invocar un modelo de chat simplemente con una cadena. Cuando se pasa una cadena como entrada, se convierte en HumanMessage y luego se pasa al modelo subyacente.



```
llm.invoke("hello world")

# acaba convirtiendose en
llm.invoke(HumanMessage(content="hello world"))

```



Vamos a usar un ChatPromptTemplate para eviarselo a nuestros modelos de lenguaje.

In [ ]:
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq

# Claves de API (asegúrate de tenerlas en Secretos)
# Configurar los modelos
llm1 = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY)
llm2 = ChatGroq(model="llama-3.3-70b-versatile", api_key=GROQ_API_KEY)

# Definir plantillas de mensajes
template_sistema = """\
Eres un traductor profesional especializado en traducción técnica y literaria.
Proporciona traducciones de alta calidad y analiza tus decisiones. También sugiere alternativas para frases complejas.
"""

template_usuario = """\
Por favor, traduce este texto del español al inglés:
{texto}
"""

# Texto original para traducir
texto_original = """\
La traducción técnica requiere precisión extrema, pero también la sensibilidad para adaptarse a contextos culturales variados.
"""

# Crear el ChatPromptTemplate
chat_prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(template_sistema),
    HumanMessagePromptTemplate.from_template(template_usuario),
])

# o de esta forma, mas legible...

chat_prompt_template = ChatPromptTemplate.from_messages([
    ("system", template_sistema),
    ("human", template_usuario),
])

prompt = chat_prompt_template.format(texto=texto_original)

# Mostramos el prompt de mensajes construido
print("El prompt de mensajes enviado es:")
print(prompt)

# Invocamos los modelos, pasandoles los mensajes
respuesta_openai = llm1.invoke(prompt)
respuesta_groq = llm2.invoke(prompt)

# Mostramos los resultados
print("\nTraducción y análisis con ChatOpenAI:")
print(respuesta_openai.content)

print("\nTraducción y análisis con ChatGroq:")
print(respuesta_groq.content)



### Reto 3: Análisis de Traducciones con Modelos de Lenguaje

En este reto, tu tarea es desarrollar un programa que utilice un modelo de lenguaje (LLM) para analizar una traducción proporcionada, identificar errores comunes y sugerir correcciones. Este ejercicio se centra en el uso de herramientas como `ChatPromptTemplate` para interactuar con el modelo, configurando un contexto claro y proporcionando instrucciones específicas.

#### Objetivo:

El programa deberá:

1. **Configurar el contexto del modelo**:
    
    - El modelo debe actuar como un experto en revisión de traducciones, especializado en identificar errores lingüísticos comunes.
2. **Procesar los datos de entrada**:
    
    - Incluir dos textos: uno original en español y su correspondiente traducción al inglés.
    - Los textos pueden contener errores deliberados, como falsos cognados, estructuras gramaticales incorrectas o problemas de terminología.
3. **Realizar el análisis**:
    
    - Utilizar el LLM para:
        - Detectar errores en la traducción.
        - Sugerir correcciones claras para cada error identificado.
        - Explicar brevemente cada decisión de corrección.
4. **Mostrar los resultados**:
    
    - Presentar un informe que incluya:
        - Los errores identificados.
        - Las correcciones sugeridas.
        - Una breve justificación de las decisiones tomadas.

#### Requisitos Técnicos:

- Utilizar `ChatPromptTemplate` para configurar los mensajes del sistema y del usuario.
- Configurar un modelo como `ChatOpenAI` o equivalente para realizar la tarea.
- Garantizar que el contexto y las instrucciones proporcionadas al modelo sean claras y específicas.

#### Entrada del Programa:

- **Texto original** (en español): Un párrafo breve que actúe como base para la traducción.
- **Texto traducido** (en inglés): La traducción del texto original, que puede contener errores intencionados.

#### Salida Esperada:

- Un análisis detallado en el que se muestren:
    - Los errores identificados (e.g., falsos cognados, problemas de fluidez o precisión).
    - Las correcciones sugeridas para cada error.
    - Explicaciones justificando las correcciones realizadas.

#### Ejemplo:

Entrada:

- Texto original (Español):  
    "El director de la empresa estaba consternado por la situación, pero expresó su determinación de buscar soluciones."
    
- Traducción (Inglés):  
    "The director of the company was constipated about the situation, but expressed his determination to look for solutions."
    

Salida esperada:

1. **Error**: "constipated" no es una traducción adecuada de "consternado".
    
    - **Corrección sugerida**: "dismayed".
    - **Explicación**: "Constipated" es un falso cognado y se refiere a un problema físico, no emocional.
2. **Error**: Problema menor en la frase "look for solutions" que podría ser más formal.
    
    - **Corrección sugerida**: "seek solutions".
    - **Explicación**: "Seek" es un término más formal y adecuado para un contexto empresarial.


### Solución Reto 3: Análisis de Traducciones con Modelos de Lenguaje

In [ ]:
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq

# Claves de API (asegúrate de tenerlas en Secretos)
# Configurar los modelos
llm1 = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY)
llm2 = ChatGroq(model="llama-3.3-70b-versatile", api_key=GROQ_API_KEY)

# Definir el reto
print("Reto de programación:")
print("Desarrolla un programa que utilice un LLM para analizar una traducción proporcionada, "
      "identificar posibles errores y sugerir correcciones.")

# Definir las plantillas
template_sistema = """\
Eres un experto en traducción especializado en revisar textos traducidos.
Identificas errores comunes como falsos cognados, estructuras gramaticales incorrectas y falta de precisión terminológica.
Proporciona sugerencias claras para corregirlos.
"""

template_usuario = """\
Aquí tienes una traducción para analizar:

Texto original (Español):
{texto_original}

Traducción (Inglés):
{texto_traducido}

Por favor, analiza los errores comunes y sugiere correcciones.
"""

# Texto de ejemplo
texto_original = """\
El director de la empresa estaba consternado por la situación, pero expresó su determinación de buscar soluciones.
"""
texto_traducido = """\
The director of the company was constipated about the situation, but expressed his determination to look for solutions.
"""

# Crear el ChatPromptTemplate
chat_prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(template_sistema),
    HumanMessagePromptTemplate.from_template(template_usuario),
])

# Formatear el prompt con los textos
prompt = chat_prompt_template.format(
    texto_original=texto_original,
    texto_traducido=texto_traducido
)

# Mostramos el prompt de mensajes construido
print("El prompt de mensajes enviado es:")
print(prompt)

# Invocamos los modelos, pasandoles los mensajes
respuesta_openai = llm1.invoke(prompt)
respuesta_groq = llm2.invoke(prompt)

# Mostramos los resultados
print("\nTraducción y análisis con ChatOpenAI:")
print(respuesta_openai.content)

print("\nTraducción y análisis con ChatGroq:")
print(respuesta_groq.content)


### 6. PromptTemplate vs ChatPromptTemplate ¿ Cuando usar uno u otro ?
---

La diferencia principal entre `PromptTemplate` y `ChatPromptTemplate` en LangChain radica en el tipo de modelos con los que están diseñados para trabajar y en cómo estructuran los mensajes.

### **PromptTemplate**

#### Uso y Propósito:

- **Diseñado para modelos tradicionales** como los basados en texto (e.g., GPT-3, `text-davinci`).
- Se utiliza para construir un único bloque de texto a partir de una plantilla.
- Ideal para modelos que aceptan una sola entrada textual como prompt.

#### Estructura:

Un `PromptTemplate` toma variables y las inserta en un solo texto formateado.

### **ChatPromptTemplate**

#### Uso y Propósito:

- **Diseñado para modelos conversacionales** como los de la familia `chat` (e.g., GPT-4 Turbo, ChatGPT).
- Permite estructurar mensajes en varios niveles, especificando roles como `system`, `user`, y `assistant`.
- Ideal para contextos donde necesitas construir conversaciones completas con múltiples mensajes.

#### Estructura:

Un `ChatPromptTemplate` organiza varios mensajes en una lista, cada uno con un rol específico.

### **Recomendación**

- **Si estás trabajando con modelos de chat**, utiliza `ChatPromptTemplate` para aprovechar su capacidad de estructurar roles.
- **Si necesitas algo rápido y sencillo**, o trabajas con modelos de texto, `PromptTemplate` será suficiente.

En el caso del reto, usar `ChatPromptTemplate` sería más adecuado porque permite organizar el análisis en roles (sistema para contexto, usuario para instrucciones).  

La calidad de los resultados podría verse afectada al usar un `PromptTemplate` en lugar de un `ChatPromptTemplate` si el modelo está optimizado para manejar roles (como los modelos de chat). Esto se debe a que los roles estructurados ayudan a organizar el contexto de forma más clara, lo que puede influir en cómo el modelo interpreta y responde a las instrucciones.

Al estar estos modelos diseñados para trabajar con roles explícitos, como `system`, `user`, y `assistant`.
    - Usar roles estructurados permite:
        - Separar instrucciones contextuales (e.g., "Eres un traductor experto").
        - Clarificar las interacciones del usuario (e.g., "Traduce este texto").
        - Mejorar la precisión y relevancia de las respuestas.

**Si usas `PromptTemplate` aquí**, podrías perder claridad en el contexto, lo que puede dar lugar a respuestas menos precisas o más genéricas.

Por ejemplo el **Reto 3: Análisis de Traducciones con Modelos de Lenguaje** prodria haberse resuelto con cualquiera de ambas clases.

Solo habria variado la forma de estructurar el prompt:

**Con `PromptTemplate`**:

- Todo el contexto y las instrucciones se construyen en un solo bloque de texto.
- El prompt podría quedar así
```
template = '''\
Eres un traductor profesional especializado en traducción técnica y literaria.
Identifica errores en la siguiente traducción y sugiere correcciones:
Texto original: {texto_original}
Traducción: {texto_traducido}
Por favor, analiza errores y explica las decisiones tomadas.'''

**Con `ChatPromptTemplate:`**

- Se separan las instrucciones del sistema y las del usuario en distintos roles.
- El codigo se veria asi:  

```
SystemMessagePromptTemplate.from_template(
    "Eres un traductor profesional especializado en identificar errores en traducciones."
)
HumanMessagePromptTemplate.from_template(
    "Texto original: {texto_original}\nTraducción: {texto_traducido}\nPor favor, analiza errores y explica tus decisiones."
)  
```

En el Playground de OpenAI puede ver los modelos antiguos (que ya estan marcados como Legacy, GPT 3.5, Davinci-002, ... ) en el apartado Completions. https://platform.openai.com/playground/complete frente a los modelos nuevos (GPT-4o, ... ) en el apartado chat https://platform.openai.com/playground/chat

## **7. Agregando más interacción: AIMessagePromptTemplate**
A veces queremos incluir un mensaje del assistant (IA) en el contexto. Esto puede ser útil para simular una conversación previa o para inyectar algún contenido que la IA haya dicho antes.  

De esta forma, el modelo interpretará que ya hubo un mensaje previo de la IA y continuará la conversación.  

<br><br>

Este ejemplo muestra un caso práctico del uso de AIMessagePromptTemplate donde:

1. Incluimos feedback de IA como parte del contexto del prompt, simulando un diálogo más natural  

2. El feedback de la IA sirve como guía para mejorar la traducción original  

3. Se mantiene una conversación estructurada: sistema → usuario → IA → usuario

La diferencia clave con el ejemplo original es que aquí usamos AIMessagePromptTemplate para incorporar retroalimentación específica de la IA como parte del contexto, lo que ayuda a obtener una traducción más refinada y consciente de aspectos específicos a mejorar.

In [ ]:
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate
from langchain_openai import ChatOpenAI

# Configurar el modelo
llm = ChatOpenAI(model="gpt-4o", api_key=OPENAI_API_KEY)

# Definir plantillas de mensajes
template_sistema = """\
Eres un tutor de idiomas especializado en mejorar traducciones basándote en feedback previo.
Analiza la traducción original y el feedback proporcionado para sugerir una versión mejorada.
"""

template_usuario_inicial = """\
Traduce esta frase al inglés:
{texto}
"""

# Este es un ejemplo de feedback de IA que se utilizará como parte del contexto
template_ai_feedback = """\
La traducción es gramaticalmente correcta, pero podría mejorar en:
1. Uso de terminología más específica del campo
2. Mayor naturalidad en la expresión
3. Consideración del contexto cultural
"""

template_usuario_final = """\
Por favor, proporciona una versión mejorada considerando este feedback.
"""

# Crear el prompt con múltiples mensajes incluyendo el feedback de la IA
chat_prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(template_sistema),
    HumanMessagePromptTemplate.from_template(template_usuario_inicial),
    AIMessagePromptTemplate.from_template(template_ai_feedback),
    HumanMessagePromptTemplate.from_template(template_usuario_final)
])

# Texto para traducir
texto_original = """\
El software debe ser compatible con los principales sistemas operativos y
ofrecer una interfaz intuitiva para usuarios principiantes.
"""

# Generar el prompt completo
prompt = chat_prompt_template.format(texto=texto_original)

# Mostrar el prompt construido
print("El prompt de mensajes enviado es:")
print(prompt)

# Invocar el modelo
respuesta = llm.invoke(prompt)

# Mostrar el resultado
print("\nTraducción mejorada y análisis:")
print(respuesta.content)


En teoría sí se puede lograr algo similar incorporando el feedback directamente en el mensaje del sistema o del usuario, pero hay algunas consideraciones importantes:

Las principales diferencias y consideraciones son:

1. **Estructura del diálogo**:
    - Con AIMessagePromptTemplate teníamos una estructura más natural de conversación que simula un proceso de revisión real.
    - Sin él, tenemos que incluir todas las instrucciones de una vez, lo que puede hacer el prompt menos natural.
2. **Flexibilidad**:
    - La versión con AIMessagePromptTemplate permite mayor flexibilidad para modificar el feedback específico para cada caso.
    - La versión sin él tiene los criterios fijos en el mensaje del sistema.
3. **Contexto**:
    - Con AIMessagePromptTemplate podemos mantener un contexto más claro de "primera traducción → feedback → mejora"
    - Sin él, tenemos que pedir todo en un solo paso.
4. **Resultados**:
    - Los resultados pueden ser similares en calidad, pero la versión con AIMessagePromptTemplate suele producir respuestas más estructuradas y cercanas a un diálogo real de revisión.
    - La versión sin él depende más de cómo estructuremos las instrucciones en el prompt.

El AIMessagePromptTemplate es especialmente útil cuando:

- Necesitas simular un diálogo real con múltiples pasos
- Quieres mantener un contexto claro de retroalimentación

FALTA EL TEMA DE LOS CHATPROMPT EN FORMATO OPENAI COMO DICCIONARIOS
{ROLE:XXX, "CONTENT": XXX}